## Agent 
Quesqu'un **agent IA**

  Les **agents d'IA** sont des systèmes logiciels qui utilisent l'IA pour atteindre des objectifs et effectuer des tâches au nom des utilisateurs. Ils font preuve de raisonnement, de planification et de mémoire, et disposent d'un certain niveau d'autonomie pour prendre des décisions, apprendre et s'adapter.

  Leurs capacités sont en grande partie rendues possibles par la **capacité multimodale** de **l'IA générative** et des modèles de fondation d'IA. Les agents d'IA peuvent traiter simultanément des informations multimodales telles que du texte, de la voix, des vidéos, des sons, du code, etc. Ils peuvent converser, raisonner, apprendre et prendre des décisions. Ils peuvent apprendre au fil du temps et faciliter les transactions et les processus métier. Les agents peuvent collaborer avec d'autres agents pour coordonner et exécuter des workflows plus complexes.

In [2]:
import os
from IPython.display import display, clear_output, Markdown
from dotenv import load_dotenv
from datetime import datetime
from langchain_ollama import ChatOllama
from langchain_deepseek import ChatDeepSeek
from langchain import hub
from langchain_core.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory
from pprint import pprint
from langchain_ollama import OllamaEmbeddings
from langchain_core.messages import SystemMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.docstore.document import Document
from langchain_core.prompts import PromptTemplate
import sys
sys.path.append("../")
from settings import vectorizing_params, retriever_params
# Chargement des clés d'API se trouvant dans le fichier .env.  
# Ceci permet d'utiliser des modèles en ligne comme gpt-x, deepseek-x, etc...
load_dotenv(override=True)

#model = ChatOllama(model="llama3", temperature=0)
model = ChatDeepSeek(model="deepseek-chat", api_key=os.getenv("DEEPSEEK_API_KEY"))
embedder = OllamaEmbeddings(model="nomic-embed-text")

ModuleNotFoundError: No module named 'dotenv'

In [2]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, "data")
db_dir = os.path.join(current_dir, "db")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=vectorizing_params['chunk_size'],
    chunk_overlap=vectorizing_params['chunk_overlap']
)

documents = []

if not os.path.exists(db_dir):
    print("Initializing vector store...")   


    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)

                with open(file_path, "r", encoding="utf-8") as f:
                    full_text = f.read()

                # Extract themes and subthemes from the file structure
                relative_path = os.path.relpath(file_path, "data")
                parts = relative_path.split(os.sep)
                large_theme = parts[0]
                theme = parts[1]
                subtheme = parts[2].replace(".txt", "")

                # Chunk splitting
                chunks = text_splitter.split_text(full_text)
                for i, chunk in enumerate(chunks):
                    documents.append(
                        Document(
                            page_content=chunk,
                            metadata={
                                "large_theme": large_theme,
                                "theme": theme,
                                "subtheme": subtheme,
                                "chunk_id": i,
                                "source": file_path
                            }
                        )
                    )

    vectorstore = Chroma.from_documents(
        documents,
        embedding=embedder,
        collection_name="droits",
        persist_directory=db_dir
    )
else:
    vectorstore = Chroma(
        persist_directory=db_dir,
        embedding_function=embedder,
        collection_name="droits"
    )

#db = Chroma.from_documents(chunks, embedder, persist_directory=db_dir)

print(f"Vectorstore created with {len(documents)} chunks.")

#db = Chroma(persist_directory=db_dir, embedding_function=embedder)

retriever = vectorstore.as_retriever(
    search_type=retriever_params['search_type'],
    search_kwargs=retriever_params['search_kwargs']
)

Vectorstore created with 0 chunks.


In [3]:
qa_chain = ConversationalRetrievalChain.from_llm(llm=model, retriever=retriever)

chat_history = [
    SystemMessage(content="Tu es un assistant qui aide à trouver des informations concernant les droits disponibles.")
]
def _format_docs(docs):
    
    result = "\n".join(
        [
            f'<item index="{i+1}">\n<page_content>\n{r}\n</page_content>\n</item>'
            for i, r in enumerate(docs)
        ]
    )
    return result

def format_agent_scratchpad(intermediate_steps):
    thoughts = ""
    for action, observation in intermediate_steps:
        thoughts += action.log
        thoughts += "</search_query>" + _format_docs(observation)
    return thoughts

def ask_rag(query: str) -> str:

    relevant_chunks = retriever.invoke(query)
            
    input_message = (
        "Voici des documents qui vont t'aider à répondre à la question : "
        + query
        + "\n\nDocuments pertinents : \n"
        + "\n\n".join([chunk.page_content for chunk in relevant_chunks])
        + "\n\nDonne une réponse basée uniquement sur les documents qui te sont fournis."
    )

    result = qa_chain({
        "question": input_message,
        "chat_history": chat_history
    })
    chat_history.append((query, result["answer"]))
    return result["answer"]

rag_tool = Tool(
    name="consult_droit",
    func=ask_rag,
    description="Répond à des questions sur les droits sociaux (APL, RSA, etc.). Fournit des réponses fiables extraites de documents organisés par thème."
)
tools=[
    rag_tool
]

tool_names="consult_droit"

agent_scratchpad=chat_history

## Prompt

le choix ce porte pour **rlm/rag-prompt** car il est plus adapter pour le chat les question/réponse.


  source : https://smith.langchain.com/hub/rlm/rag-prompt

In [7]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Role: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.If your source is from internet use italic and bold font.
Use three sentences maximum and keep the answer concise.

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 5 times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [8]:
# Chargement du prompt standard pour le paradigme ReAct depuis LangChain Hub
#prompt = hub.pull("hwchase17/react-chat")
# Initialisation de la mémoire pour suivre l’historique des échanges
memory= ConversationBufferMemory(memory_key="chat_history", return_messages=True)
#creation agent
agent=create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
    stop_sequence=True
    
)

#encapsulation agent
#enlever dans la version final mettre Verbose a false
executor=AgentExecutor.from_agent_and_tools( 
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True
)

In [9]:
# Boucle interactive terminale
while True:
    user_input = input("Vous : ")
    clear_output(wait=True)                         # Efface l'affichage précédent
    display(Markdown(f"**Vous :** {user_input}"))   # Affiche la requête de l'utilisateur

    if user_input.lower() in ["stop", "exit", "quit"]:
        print("Fin de la conversation.")
        break

    response = executor.invoke({"input": user_input})
    display(Markdown(response["output"]))

**Vous :** quit

Fin de la conversation.
